# Building Machine Learning Classifiers: Explore Gradient Boosting model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [2]:
import sys
!{sys.executable} -m pip install nltk

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
!nltk

In [9]:
#import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore GradientBoostingClassifier Attributes & Hyperparameters

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_initialized', '_check_n_features', '_check_params', '_clear_state', '_compute_partial_dependence_recursion', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_more_tags', '_raw_predict', '_raw_predict_init', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_data', '_validate_estimator', '_validate_y', '_warn_mae_for_criterion', 'apply

### Build our own Grid-search

In [12]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [20]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, 
                                   learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    
    #must give 4 objects for score to write out to
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('''
    Est: {} \ Depth: {} \ Learning Rate: {} ---- Precision: {} \ Recall: {} \ Accuracy: {}'''
          .format(est, max_depth, lr, 
                  precision.round(3), 
                  recall.round(3), 
                  ((y_pred == y_test).sum()/len(y_pred)).round()))

In [21]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\Aim\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



    Est: 50 \ Depth: 3 \ Learning Rate: 0.01 ---- Precision: 0.0 \ Recall: 0.0 \ Accuracy: 1.0

    Est: 50 \ Depth: 3 \ Learning Rate: 0.1 ---- Precision: 0.894 \ Recall: 0.697 \ Accuracy: 1.0

    Est: 50 \ Depth: 3 \ Learning Rate: 1 ---- Precision: 0.844 \ Recall: 0.786 \ Accuracy: 1.0


C:\Users\Aim\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



    Est: 50 \ Depth: 7 \ Learning Rate: 0.01 ---- Precision: 0.0 \ Recall: 0.0 \ Accuracy: 1.0

    Est: 50 \ Depth: 7 \ Learning Rate: 0.1 ---- Precision: 0.865 \ Recall: 0.793 \ Accuracy: 1.0

    Est: 50 \ Depth: 7 \ Learning Rate: 1 ---- Precision: 0.874 \ Recall: 0.766 \ Accuracy: 1.0


C:\Users\Aim\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



    Est: 50 \ Depth: 11 \ Learning Rate: 0.01 ---- Precision: 0.0 \ Recall: 0.0 \ Accuracy: 1.0

    Est: 50 \ Depth: 11 \ Learning Rate: 0.1 ---- Precision: 0.873 \ Recall: 0.807 \ Accuracy: 1.0

    Est: 50 \ Depth: 11 \ Learning Rate: 1 ---- Precision: 0.891 \ Recall: 0.786 \ Accuracy: 1.0


C:\Users\Aim\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



    Est: 50 \ Depth: 15 \ Learning Rate: 0.01 ---- Precision: 0.0 \ Recall: 0.0 \ Accuracy: 1.0

    Est: 50 \ Depth: 15 \ Learning Rate: 0.1 ---- Precision: 0.838 \ Recall: 0.821 \ Accuracy: 1.0

    Est: 50 \ Depth: 15 \ Learning Rate: 1 ---- Precision: 0.874 \ Recall: 0.814 \ Accuracy: 1.0

    Est: 100 \ Depth: 3 \ Learning Rate: 0.01 ---- Precision: 0.878 \ Recall: 0.497 \ Accuracy: 1.0

    Est: 100 \ Depth: 3 \ Learning Rate: 0.1 ---- Precision: 0.905 \ Recall: 0.786 \ Accuracy: 1.0

    Est: 100 \ Depth: 3 \ Learning Rate: 1 ---- Precision: 0.865 \ Recall: 0.793 \ Accuracy: 1.0

    Est: 100 \ Depth: 7 \ Learning Rate: 0.01 ---- Precision: 0.912 \ Recall: 0.641 \ Accuracy: 1.0

    Est: 100 \ Depth: 7 \ Learning Rate: 0.1 ---- Precision: 0.889 \ Recall: 0.828 \ Accuracy: 1.0

    Est: 100 \ Depth: 7 \ Learning Rate: 1 ---- Precision: 0.856 \ Recall: 0.779 \ Accuracy: 1.0

    Est: 100 \ Depth: 11 \ Learning Rate: 0.01 ---- Precision: 0.912 \ Recall: 0.71 \ Accuracy: 1.0


KeyboardInterrupt: 